# Paso 1: Configuración del Entorno

In [0]:
%pip install beautifulsoup4 requests pandas

Python interpreter will be restarted.
Python interpreter will be restarted.


# Importar librerías

In [0]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

# Paso 2: Clase DataWeb

### Clase especializada para extraer datos financieros de Yahoo Finance

In [0]:




class DataWeb:
    def __init__(self):
        self.url = "https://es.finance.yahoo.com/quote/DOLA-USD/history/"
    

    def obtener_datos(self):
        try:
            # url , cabeceras
            headers = {
                'User-Agent': 'Mozilla/5.0'
            }
            respuesta = requests.get(self.url,headers=headers)
            if respuesta.status_code != 200:
                print("La url saco error, no respondio o no existe")
            #print(respuesta.text)
            soup = BeautifulSoup(respuesta.text,'html.parser')
            tabla = soup.select_one('div[data-testid="history-table"] table')
            nombre_columnas = [th.get_text(strip=True) for th in tabla.thead.find_all('th')]
            filas = []
            for tr in tabla.tbody.find_all('tr'):
                columnas = [ td.get_text(strip=True) for td in tr.find_all('td')]
                if len(columnas) == len(nombre_columnas):
                    filas.append(columnas)
            df = pd.DataFrame(filas,columns=nombre_columnas).rename(columns = {
                'Fecha': 'fecha',
                'Abrir': 'abrir',
                'Máx.': 'max',
                'Mín.': 'min',
                'CerrarPrecio de cierre ajustado para splits.': 'cerrar',
                'Cierre ajustadoPrecio de cierre ajustado para splits y distribuciones de dividendos o plusvalías.': 'cierre_ajustado',
                'Volumen':'volumen'
            })
            df = self.convertir_numericos(df)
            #df.to_excel("dataweb_limpio.xlsx")
            # print(nombre_columnas)
            # print(filas)
            # for th in tabla.thead.find_all('th'):
            #     th_data = th.get_text(strip=True)
# ['Fecha', 'Abrir', 'Máx.', 'Mín.', 'CerrarPrecio de cierre ajustado para splits.', 'Cierre ajustadoPrecio de cierre ajustado para splits y distribuciones de dividendos o plusvalías.', 'Volumen']

            return df
        except Exception as err:
            print("Error en la funcion obtener_datos")
            df = pd.DataFrame()
            return df


    def convertir_numericos(self,df=pd.DataFrame()):
        df= df.copy()
        if len(df)>0:
            #for col in (df.columns):
            for col in ('abrir',	'max',	'min',	'cerrar',	'cierre_ajustado',	'volumen'):
                df[col] = (df[col]
                           .str.replace(r"\.","",regex=True)
                           .str.replace(",",'.'))

        return df

# Paso 3: Crear Esquema y Tablas Delta

In [0]:
%sql
DROP TABLE IF EXISTS finanzas_schema.indicador_parametros;
DROP TABLE IF EXISTS finanzas_schema.indicador_financiero;
DROP TABLE IF EXISTS finanzas_schema.indicador_demografico;


In [0]:
%sql
DROP SCHEMA IF EXISTS finanzas_schema CASCADE;

In [0]:
%sql
-- Crear esquema principal
CREATE SCHEMA IF NOT EXISTS finanzas_schema
COMMENT 'Esquema para datos financieros y análisis de mercados';

In [0]:
%sql
-- TABLA 1: PARÁMETROS DE INDICADORES
-- =================================================
DROP TABLE IF EXISTS finanzas_schema.indicador_parametros;



In [0]:
%sql
CREATE TABLE IF NOT EXISTS finanzas_schema.indicador_parametros (
    cod_indicador STRING NOT NULL,
    descripcion STRING,
    prioridad INT,
    patron_clasificacion STRING,
    activo BOOLEAN,
    fecha_creacion TIMESTAMP
)
USING DELTA
COMMENT 'Tabla de configuración y patrones para clasificación de instrumentos financieros'
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.autoOptimize.autoCompact' = 'true'
);

In [0]:
%sql
select * from finanzas_schema.indicador_parametros ;

cod_indicador,descripcion,prioridad,patron_clasificacion,activo,fecha_creacion


In [0]:
%sql
SHOW CREATE TABLE finanzas_schema.indicador_parametros;

createtab_stmt
"CREATE TABLE spark_catalog.finanzas_schema.indicador_parametros ( cod_indicador STRING NOT NULL, descripcion STRING, prioridad INT, patron_clasificacion STRING, activo BOOLEAN, fecha_creacion TIMESTAMP) USING delta COMMENT 'Tabla de configuración y patrones para clasificación de instrumentos financieros' TBLPROPERTIES ( 'delta.autoOptimize.autoCompact' = 'true', 'delta.autoOptimize.optimizeWrite' = 'true', 'delta.minReaderVersion' = '1', 'delta.minWriterVersion' = '2')"


In [0]:
%sql
SHOW TABLES IN finanzas_schema;

database,tableName,isTemporary
finanzas_schema,indicador_parametros,false


In [0]:
%sql
-- TABLA 1: DATOS HISTÓRICOS FINANCIEROS

CREATE TABLE IF NOT EXISTS finanzas_schema.indicador_financiero (
    fecha STRING,
    abrir DOUBLE,
    max DOUBLE,
    min DOUBLE,
    cerrar DOUBLE,
    cierre_ajustado DOUBLE,
    volumen BIGINT,
    cod_indicador STRING,
    year INT,
    month INT,
    day INT,
    year_month STRING
    --,fecha_extraccion TIMESTAMP

)
--USING DELTA
--LOCATION 'dbfs:/delta/finanzas/indicador_financiero'
PARTITIONED BY (cod_indicador)
COMMENT 'Tabla principal con datos históricos de instrumentos financieros'
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.autoOptimize.autoCompact' = 'true'
)
;

In [0]:
%sql
SHOW TABLES IN finanzas_schema;

database,tableName,isTemporary
finanzas_schema,indicador_financiero,false
finanzas_schema,indicador_parametros,false


In [0]:
%sql
SHOW TABLES IN finanzas_schema;

database,tableName,isTemporary
finanzas_schema,indicador_financiero,false
finanzas_schema,indicador_parametros,false


In [0]:
%sql
DESCRIBE EXTENDED finanzas_schema.indicador_parametros;

col_name,data_type,comment
cod_indicador,string,null
descripcion,string,null
prioridad,int,null
patron_clasificacion,string,null
activo,boolean,null
fecha_creacion,timestamp,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,finanzas_schema,


In [0]:
%sql
DESCRIBE EXTENDED finanzas_schema.indicador_financiero;


col_name,data_type,comment
fecha,string,null
abrir,double,null
max,double,null
min,double,null
cerrar,double,null
cierre_ajustado,double,null
volumen,bigint,null
cod_indicador,string,null
year,int,null
month,int,null


### # Paso 4: Poblar Tabla de Parámetros

In [0]:
%sql
-- =================================================
-- POBLAR TABLA DE PARÁMETROS
-- =================================================
INSERT INTO finanzas_schema.indicador_parametros VALUES
-- ÍNDICES BURSÁTILES
('^STOXX50E', 'Euro Stoxx 50 - Índice de las 50 principales empresas de la eurozona', 1, 'INDICE_EUROPEO', true, current_timestamp()),
('^IBEX', 'IBEX 35 - Índice principal de la bolsa española', 1, 'INDICE_ESPAÑOL', true, current_timestamp()),
('^FCHI', 'CAC 40 - Índice de la bolsa de París', 1, 'INDICE_FRANCES', true, current_timestamp()),

-- COMMODITIES
('GC=F', 'Oro - Futuros del metal precioso', 1, 'COMMODITY_METAL', true, current_timestamp()),
('CL=F', 'Petróleo WTI - Futuros de crudo', 1, 'COMMODITY_ENERGIA', true, current_timestamp()),

-- DIVISAS
('EURUSD=X', 'Euro / Dólar estadounidense', 1, 'DIVISA_MAYOR', true, current_timestamp()),

-- CRIPTOMONEDAS
('BTC-EUR', 'Bitcoin en euros', 1, 'CRYPTO_MAYOR', true, current_timestamp()),

-- ACCIONES ESPAÑOLAS (muestra)
('SAN.MC', 'Banco Santander - Acción española', 2, 'ACCION_ESPAÑOLA_BANCARIA', true, current_timestamp()),
('TEF.MC', 'Telefónica - Acción española de telecomunicaciones', 2, 'ACCION_ESPAÑOLA_TELCO', true, current_timestamp());

num_affected_rows,num_inserted_rows
9,9


In [0]:
%sql
SELECT 'Parámetros insertados:' as info, COUNT(*) as cantidad 
FROM finanzas_schema.indicador_parametros;

-- Mostrar configuración
SELECT 
    cod_indicador, 
    descripcion, 
    patron_clasificacion, 
    prioridad,
    activo
FROM finanzas_schema.indicador_parametros 
ORDER BY prioridad, cod_indicador;

cod_indicador,descripcion,patron_clasificacion,prioridad,activo
BTC-EUR,Bitcoin en euros,CRYPTO_MAYOR,1,true
CL=F,Petróleo WTI - Futuros de crudo,COMMODITY_ENERGIA,1,true
EURUSD=X,Euro / Dólar estadounidense,DIVISA_MAYOR,1,true
GC=F,Oro - Futuros del metal precioso,COMMODITY_METAL,1,true
^FCHI,CAC 40 - Índice de la bolsa de París,INDICE_FRANCES,1,true
^IBEX,IBEX 35 - Índice principal de la bolsa española,INDICE_ESPAÑOL,1,true
^STOXX50E,Euro Stoxx 50 - Índice de las 50 principales empresas de la eurozona,INDICE_EUROPEO,1,true
SAN.MC,Banco Santander - Acción española,ACCION_ESPAÑOLA_BANCARIA,2,true
TEF.MC,Telefónica - Acción española de telecomunicaciones,ACCION_ESPAÑOLA_TELCO,2,true


# Paso 5: poblar tabla indicador_financiero

In [0]:

dataweb = DataWeb()
df = dataweb.obtener_datos()
df = dataweb.convertir_numericos(df)
df["cod_indicador"] ="EURUSD=X"
#DOLA-USD






In [0]:
display(df)

fecha,abrir,max,min,cerrar,cierre_ajustado,volumen,cod_indicador
29 may 2025,0999747,0999823,0999701,0999823,0999823,136497,EURUSD=X
27 may 2025,1000068,1000160,0999615,0999632,0999632,31042,EURUSD=X
26 may 2025,1000004,1000079,0999092,1000068,1000068,35141,EURUSD=X
25 may 2025,1000005,1000181,0999808,1000004,1000004,47218,EURUSD=X
24 may 2025,0999847,1000292,0999843,1000005,1000005,445929,EURUSD=X
23 may 2025,0999324,1000615,0997860,0999847,0999847,2109145,EURUSD=X
22 may 2025,1000535,1000690,0998851,0999324,0999324,148963,EURUSD=X
21 may 2025,0999983,1001080,0999914,1000535,1000535,83405,EURUSD=X
20 may 2025,1000457,1000477,0999848,0999983,0999983,54084,EURUSD=X
19 may 2025,1000716,1000737,1000101,1000457,1000457,179724,EURUSD=X


In [0]:
def completar_columnas(df, columna_fecha):
      try:
          # Extraer componentes y convertirlos explícitamente a enteros
          df['year'] = df['fecha_dt'].dt.year.astype('Int64')  # Tipo Int64 de pandas maneja NaN
          df['month'] = df['fecha_dt'].dt.month.astype('Int64')
          df['day'] = df['fecha_dt'].dt.day.astype('Int64')
          
          # Agregar columna year_month en formato yyyy-mm
          df['year_month'] = df['fecha_dt'].dt.strftime('%Y-%m')
          
          # Eliminar columna temporal
          df.drop('fecha_dt', axis=1, inplace=True)
          return df
          
      except Exception as e:
          # En caso de error, intentamos al menos devolver el DataFrame sin modificar
          if 'fecha_dt' in df.columns:
              df.drop('fecha_dt', axis=1, inplace=True)
          return df
  
def formatear_fechas(df, columna_fecha='fecha'):
      """
      Convierte el formato de fecha de '1 abr 2004' o '07 may 2025' a 'yyyy-mm-dd' 
      y agrega columnas year, month, day
      
      Parámetros:
      df (pandas.DataFrame): DataFrame que contiene la columna de fechas
      columna_fecha (str): Nombre de la columna que contiene las fechas
      
      Retorna:
      pandas.DataFrame: DataFrame actualizado
      """
      try:
          df_resultado = df.copy()
          
          # Diccionario de mapeo de meses en español a números
          meses = {
              'ene': '01', 'feb': '02', 'mar': '03', 'abr': '04', 'may': '05', 'jun': '06',
              'jul': '07', 'ago': '08', 'sept': '09', 'oct': '10', 'nov': '11', 'dic': '12'
          }
          
          def convertir_fecha(fecha_str):
              if pd.isna(fecha_str) or not isinstance(fecha_str, str):
                  return None
                  
              # Limpiar la fecha (eliminar comillas y espacios adicionales)
              fecha_str = fecha_str.strip('"\'').strip()
              
              # Extraer partes de la fecha
              partes = fecha_str.split()
              if len(partes) != 3:
                  return None
                  
              dia, mes_abr, año = partes
              mes_abr = mes_abr.lower()
              
              # Verificar si el mes está en nuestro diccionario
              if mes_abr in meses:
                  mes = meses[mes_abr]
                  # Formatear día con ceros a la izquierda si es necesario
                  dia = dia.zfill(2)
                  # Construir fecha en formato ISO (yyyy-mm-dd)
                  return f"{año}-{mes}-{dia}"
              
              return None
          
          # Aplicar la función de conversión a la columna de fechas
          df_resultado[columna_fecha] = df_resultado[columna_fecha].apply(convertir_fecha)
          
          # Convertir a datetime para extraer componentes
          df_resultado['fecha_dt'] = pd.to_datetime(df_resultado[columna_fecha], errors='coerce')
          
          # Completar columnas year, month y day
          return completar_columnas(df_resultado, columna_fecha)
          
      except Exception as e:
          print("error {}".format(e))
          return df

In [0]:
display(df.columns)

Index(['fecha', 'abrir', 'max', 'min', 'cerrar', 'cierre_ajustado', 'volumen',
       'cod_indicador'],
      dtype='object')

In [0]:
df_2 = formatear_fechas(df,"fecha")
display(df_2)

fecha,abrir,max,min,cerrar,cierre_ajustado,volumen,cod_indicador,year,month,day,year_month
2025-05-29,0999747,0999823,0999701,0999823,0999823,136497,EURUSD=X,2025,5,29,2025-05
2025-05-27,1000068,1000160,0999615,0999632,0999632,31042,EURUSD=X,2025,5,27,2025-05
2025-05-26,1000004,1000079,0999092,1000068,1000068,35141,EURUSD=X,2025,5,26,2025-05
2025-05-25,1000005,1000181,0999808,1000004,1000004,47218,EURUSD=X,2025,5,25,2025-05
2025-05-24,0999847,1000292,0999843,1000005,1000005,445929,EURUSD=X,2025,5,24,2025-05
2025-05-23,0999324,1000615,0997860,0999847,0999847,2109145,EURUSD=X,2025,5,23,2025-05
2025-05-22,1000535,1000690,0998851,0999324,0999324,148963,EURUSD=X,2025,5,22,2025-05
2025-05-21,0999983,1001080,0999914,1000535,1000535,83405,EURUSD=X,2025,5,21,2025-05
2025-05-20,1000457,1000477,0999848,0999983,0999983,54084,EURUSD=X,2025,5,20,2025-05
2025-05-19,1000716,1000737,1000101,1000457,1000457,179724,EURUSD=X,2025,5,19,2025-05


In [0]:
def convertir_columnas_numericas_pandas(df, columnas_string=None):
 
    # Crear una copia del DataFrame para evitar modificar el original
    df_modificado = df.copy()
    
    # Convertir columnas a string
    if columnas_string:
        for columna in columnas_string:
            if columna in df.columns:
                df_modificado[columna] = df_modificado[columna].fillna(0).astype('int64')
    return df_modificado

In [0]:
columnas= ['abrir', 'max', 'min', 'cerrar', 'cierre_ajustado', 'volumen']
df_2 = convertir_columnas_numericas_pandas(df_2,columnas)
display(df_2.head())

fecha,abrir,max,min,cerrar,cierre_ajustado,volumen,cod_indicador,year,month,day,year_month
2025-05-29,999747,999823,999701,999823,999823,136497,EURUSD=X,2025,5,29,2025-05
2025-05-27,1000068,1000160,999615,999632,999632,31042,EURUSD=X,2025,5,27,2025-05
2025-05-26,1000004,1000079,999092,1000068,1000068,35141,EURUSD=X,2025,5,26,2025-05
2025-05-25,1000005,1000181,999808,1000004,1000004,47218,EURUSD=X,2025,5,25,2025-05
2025-05-24,999847,1000292,999843,1000005,1000005,445929,EURUSD=X,2025,5,24,2025-05


### Esquema y estructura de la tabla 2 (tabla web)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType, IntegerType

schema_fina = StructType([
    StructField("fecha", StringType(), False, {'comment': 'Fecha de la cotización en formato YYYY-MM-DD'}),
    StructField("abrir", LongType(), True, {'comment': 'Precio de apertura del día'}),
    StructField("max", LongType(), True, {'comment': 'Precio máximo alcanzado en el día'}),
    StructField("min", LongType(), True, {'comment': 'Precio mínimo alcanzado en el día'}),
    StructField("cerrar", LongType(), True, {'comment': 'Precio de cierre del día'}),
    StructField("cierre_ajustado", LongType(), True, {'comment': 'Precio de cierre ajustado por dividendos y splits'}),
    StructField("volumen", LongType(), True, {'comment': 'Volumen de acciones negociadas'}),
    StructField("cod_indicador", StringType(), True, {'comment': 'Código del activo financiero'}),
    StructField("year", IntegerType(), True, {'comment': 'Fecha de la cotización en formato YYYY'}),
    StructField("month", IntegerType(), True, {'comment': 'Fecha de la cotización en formato MM'}),
    StructField("day", IntegerType(), True, {'comment': 'Fecha de la cotización en formato DD'}),
    StructField("year_month", StringType(), True, {'comment': 'Fecha de la cotización en formato YYYYMM'})
    #,StructField("fecha_extraccion", TimestampType(), True, {'comment': 'Timestamp de cuando se extrajo la información'})
])


### Conertir Dataframe pandas a df Spark

In [0]:
from pyspark.sql.functions import lit, current_timestamp
df_spark = spark.createDataFrame(df_2,schema=schema_fina)
spark_df = df_spark.withColumn("fecha_extraccion", current_timestamp())
display(spark_df)

fecha,abrir,max,min,cerrar,cierre_ajustado,volumen,cod_indicador,year,month,day,year_month,fecha_extraccion
2025-05-29,999747,999823,999701,999823,999823,136497,EURUSD=X,2025,5,29,2025-05,2025-05-29T02:31:53.655+0000
2025-05-27,1000068,1000160,999615,999632,999632,31042,EURUSD=X,2025,5,27,2025-05,2025-05-29T02:31:53.655+0000
2025-05-26,1000004,1000079,999092,1000068,1000068,35141,EURUSD=X,2025,5,26,2025-05,2025-05-29T02:31:53.655+0000
2025-05-25,1000005,1000181,999808,1000004,1000004,47218,EURUSD=X,2025,5,25,2025-05,2025-05-29T02:31:53.655+0000
2025-05-24,999847,1000292,999843,1000005,1000005,445929,EURUSD=X,2025,5,24,2025-05,2025-05-29T02:31:53.655+0000
2025-05-23,999324,1000615,997860,999847,999847,2109145,EURUSD=X,2025,5,23,2025-05,2025-05-29T02:31:53.655+0000
2025-05-22,1000535,1000690,998851,999324,999324,148963,EURUSD=X,2025,5,22,2025-05,2025-05-29T02:31:53.655+0000
2025-05-21,999983,1001080,999914,1000535,1000535,83405,EURUSD=X,2025,5,21,2025-05,2025-05-29T02:31:53.655+0000
2025-05-20,1000457,1000477,999848,999983,999983,54084,EURUSD=X,2025,5,20,2025-05,2025-05-29T02:31:53.655+0000
2025-05-19,1000716,1000737,1000101,1000457,1000457,179724,EURUSD=X,2025,5,19,2025-05,2025-05-29T02:31:53.655+0000


### Insertar los datos en la tabla 2 finanzas_schema.indicador_financiero modo: insertar al final

In [0]:
spark_df.write \
    .format("parquet") \
    .mode("append") \
    .partitionBy("cod_indicador") \
    .saveAsTable("indicador_financiero")

In [0]:
display(spark.sql("SELECT * FROM datos_financieros"))

fecha,abrir,max,min,cerrar,cierre_ajustado,volumen,fecha_extraccion,cod_indicador
27 nov 2024,0994844,0995100,0987895,0990683,0990683,667196,2025-05-29T01:02:52.475+0000,EURUSD=X
26 nov 2024,0995305,1000345,0990985,0994844,0994844,895951,2025-05-29T01:02:52.475+0000,EURUSD=X
25 nov 2024,0993739,0996692,0992103,0995305,0995305,501971,2025-05-29T01:02:52.475+0000,EURUSD=X
24 nov 2024,0994740,0996388,0991215,0993739,0993739,430280,2025-05-29T01:02:52.475+0000,EURUSD=X
23 nov 2024,0994014,0998054,0982249,0994740,0994740,578592,2025-05-29T01:02:52.475+0000,EURUSD=X
22 nov 2024,0983609,0999960,0977079,0994013,0994013,5244866,2025-05-29T01:02:52.475+0000,EURUSD=X
21 nov 2024,0991677,0998963,0956393,0983611,0983611,5495438,2025-05-29T01:02:52.475+0000,EURUSD=X
20 nov 2024,0991185,0995559,0982085,0991677,0991677,1408414,2025-05-29T01:02:52.475+0000,EURUSD=X
19 nov 2024,0990920,0997699,0983738,0991185,0991185,1474654,2025-05-29T01:02:52.475+0000,EURUSD=X
18 nov 2024,0992245,1001486,0984366,0990920,0990920,1486320,2025-05-29T01:02:52.475+0000,EURUSD=X


In [0]:
display(spark.sql("DESCRIBE EXTENDED datos_financieros"))

col_name,data_type,comment
fecha,string,null
abrir,string,null
max,string,null
min,string,null
cerrar,string,null
cierre_ajustado,string,null
volumen,string,null
fecha_extraccion,timestamp,null
cod_indicador,string,null
# Partition Information,,


In [0]:
display(spark.sql("SHOW PARTITIONS datos_financieros"))

partition
cod_indicador=EURUSD%3DX
